In [1]:
%matplotlib inline
from config_calc import *
import grid_tools

import xarray as xr
import numpy as np

import cesm_orcas_sci as cesm
import cam
import gv
import xcalendar as xcal

import colorbrewer

import pandas
import statsmodels.api as sm
from scipy import stats

import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

In [2]:
case = 'bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004'
obs, mdl = gv.open_flightdata(case, mask=True)
mdl.info()

/gpfs/u/home/mclong/p/orcas/earth_geometry.py:57: RuntimeWarning: invalid value encountered in less_equal
  mask = np.array((haversine(clon,clat,plon.ravel(),plat.ravel()) <= range_km))
/gpfs/u/home/mclong/p/orcas/gv.py:48: RuntimeWarning: invalid value encountered in less
  land_mask = ~( (points_in_range(airport_lon,airport_lat,x,y,10.)) & (z < 4.) )
/gpfs/u/home/mclong/p/orcas/gv.py:51: RuntimeWarning: invalid value encountered in greater_equal
  region_mask = ( (lat_rgn[0] <= y) & (y <= lat_rgn[1]) & (lon_rgn[0] <= x) & (x <= lon_rgn[1]) )
/gpfs/u/home/mclong/p/orcas/gv.py:51: RuntimeWarning: invalid value encountered in less_equal
  region_mask = ( (lat_rgn[0] <= y) & (y <= lat_rgn[1]) & (lon_rgn[0] <= x) & (x <= lon_rgn[1]) )


xarray.Dataset {
dimensions:
	time = 34205 ;

variables:
	float64 O2_OCN(time) ;
		O2_OCN:units = per meg ;
		O2_OCN:long_name = O2_OCN ;
	float64 CO2_T09k(time) ;
		CO2_T09k:units = ppmv ;
		CO2_T09k:long_name = Takahashi (Dec x -150%) ;
	float64 aO2_GKA(time) ;
		aO2_GKA:units = per meg ;
		aO2_GKA:long_name = GK2001 (abiotic) ;
	float64 CO2_C15O(time) ;
		CO2_C15O:units = ppmv ;
		CO2_C15O:long_name = CT2015 (ocean) ;
	float64 CO2_LND(time) ;
		CO2_LND:units = ppmv ;
		CO2_LND:long_name = CO2_LND ;
	float64 U10(time) ;
		U10:units = m/s ;
		U10:long_name = 10m wind speed ;
		U10:cell_methods = time: mean ;
	float64 CO2_T09(time) ;
		CO2_T09:units = ppmv ;
		CO2_T09:long_name = Takahashi (2009) ;
	float64 PS(time) ;
		PS:units = hPa ;
		PS:long_name = Surface pressure ;
		PS:cell_methods = time: mean ;
	float64 CO2_T09b(time) ;
		CO2_T09b:units = ppmv ;
		CO2_T09b:long_name = Takahashi (Jan x +50%) ;
	float64 CO2_T09c(time) ;
		CO2_T09c:units = ppmv ;
		CO2_T09c:long_name = Takahashi

In [3]:
tracer_def = cesm.trace_gas_tracers(case)
tracer_3d = [k for k in tracer_def]

ds = cesm.open_casedata(case,'atm', 'cam.h0', tracer_3d + ['SF'+v for v in tracer_3d if 'IDL' not in v],
                        transformed='so_ocean_mean')

ds = cesm.convert_dataset(ds,case)
ds = ds.isel(zlev=np.where(ds.zlev < 15e3)[0])
ds.CO2_OCN.attrs['long_name'] = 'CESM'
ds

FileNotFoundError: [Errno 2] No such file or directory: b'/glade/p/eol/stephens/longcoll/hpss-mirror/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004/atm/proc/tseries/daily_so_ocean_mean/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004.cam.h0.O2_GKA.20070101-20160229.nc'

In [ ]:
grid = xr.open_dataset('./data/f09_f09.nc')
landfrac = grid.LANDFRAC.isel(time=0)
area = grid_tools.compute_grid_area(grid.lon.values,grid.lat.values)
rmask = landfrac.where(landfrac<0.9).fillna(0.).where(landfrac>=0.9).fillna(1.).where(landfrac.lat<-44.).fillna(0.)
plt.figure()
rmask.plot()
area = rmask * area
plt.figure()
area.plot()

### compute region integral of fluxes (from avg)

In [ ]:
co2_ocn = ['CO2_T09', 'CO2_L14C',
           'CO2_T09a', 'CO2_T09b', 'CO2_T09c', 
           'CO2_T09d', 'CO2_T09e', 'CO2_T09f', 'CO2_T09g', 
           'CO2_T09h', 'CO2_T09i', 'CO2_T09j', 'CO2_T09k', 
           'CO2_T09l','CO2_OCN']
#           'CO2_CRPO', 'CO2_CROO', 'CO2_C15O']

sfint = xr.Dataset()
for v in co2_ocn:
    sfint[v] = ds['SF'+v] * area.sum() * 12.e-15
    sfint[v].attrs['units'] = 'Pg C yr$^{-1}$'

### compute vertical gradient

In [ ]:
#lower_trop = np.array([0,2000.])
#upper_trop = np.array([3000,4000.])

lower_trop = np.array([2000.,3000.])
upper_trop = np.array([5000,6000.])


nx_lower_trop = np.where((lower_trop[0]<=ds.zlev)&(ds.zlev<=lower_trop[1]))[0]
nx_upper_trop = np.where((upper_trop[0]<=ds.zlev)&(ds.zlev<=upper_trop[1]))[0]
vg_bins = np.concatenate((lower_trop,upper_trop))


compute vertical gradient from model regional mean

In [ ]:
vg = ds.groupby_bins('zlev',vg_bins).mean(dim='zlev')
vg = vg.isel(zlev_bins=0) - vg.isel(zlev_bins=-1)
vg.info()

Observed vertical gradient

In [ ]:
vg_obs = obs.groupby_bins('GGALT',vg_bins*1e-3).mean()
vg_mdl = mdl.groupby_bins('GGALT',vg_bins*1e-3).mean()

vg_obs = vg_obs.isel(GGALT_bins=0) - vg_obs.isel(GGALT_bins=-1)
vg_mdl = vg_mdl.isel(GGALT_bins=0) - vg_mdl.isel(GGALT_bins=-1)
vg_mdl.info()

In [ ]:
tbl = {}
for v in ['CO2','CO2_OCN','CO2_LND','CO2_FFF']:
    tbl[v] = vg_mdl[v].values

tbl['CO2_NOAA'] = vg_obs.CO2_NOAA.values
tbl = pandas.DataFrame.from_dict(tbl,orient='index')
tbl

In [ ]:
vg_mdl.CO2_OCN.values - vg_mdl.CO2_FFF.values

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
wJ = 0.7
wF = 0.0
wD = 0.3

flux_weight = lambda da: (da.sel(time=slice('2015-12-01','2016-02-28')).groupby('time.month').mean()*np.array((wJ,wF,wD))).sum()

filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', 'h', 'H', 'D', 'd')
x = np.array([])
y = np.array([])
i = 0
for v in co2_ocn:
    if 'CO2_OCN' in v: continue
    xi = vg_mdl[v]
    yi = flux_weight(sfint[v])
    x = np.append(x,xi)
    y = np.append(y,yi)    
    if 'T09' in v: 
        marker = filled_markers[i]
        i += 1
        if i == len(filled_markers): i = 0
    else:
        marker = filled_markers[0]
    ax.plot(xi,yi,'.',label=ds[v].long_name,marker=marker,markersize=4)

    
ax.plot(vg_mdl.CO2_OCN,flux_weight(sfint.CO2_OCN),'*',markersize=6,label='CESM')


handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles, labels,bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

model = sm.OLS(y, sm.add_constant(x))
fitted = model.fit()

xhat = np.linspace(x.min(),x.max(),50)
yhat = fitted.predict(sm.add_constant(xhat))
yerr = y - fitted.predict(sm.add_constant(x))
xbar = np.mean(x)
n = len(x)
df = n - fitted.df_model - 1
t = stats.t.ppf(1.-0.025,df=df)

print(fitted.params)
print(fitted.summary())
s_err = np.sum(np.power(yerr,2))
ci = t * np.sqrt((s_err/(n-2))*(1.0/n + (np.power((xhat-xbar),2) / 
((np.sum(np.power(xhat,2))) - n*(np.power(xbar,2))))))

ax.plot(xhat,yhat,'-',linewidth=1,color='k',zorder=-10)    

ax.axvline(vg_obs.CO2_NOAA-vg_mdl.CO2_FFF-vg_mdl.CO2_LND,color='k')
ax.text(vg_obs.CO2_NOAA-vg_mdl.CO2_FFF-vg_mdl.CO2_LND+0.003,-0.8,'Observed\ngradient\n(corrected for fossil\nand land)')

ax.text(-0.3,-1.4,'{0:0.2f} Pg C yr$^{{-1}}$:ppm\nr$^2$={1:0.3f}'.format(fitted.params[1],fitted.rsquared))

ax.set_xlabel('Vertical CO$_2$ gradient [ppm]')
ax.set_ylabel('Southern Ocean CO$_2$ flux [Pg C yr$^{-1}$]')
plt.savefig(os.path.join(diro['fig'],'calibrate-vertical-gradient-orcas.pdf'),bbox_extra_artists=(lgd,),bbox_inches='tight')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
#wJ = 0.65
#wF = 0.0
#wD = 0.35

filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', 'h', 'H', 'D', 'd')
x = np.array([])
y = np.array([])
i = 0
for v in co2_ocn:
    xi = np.array([])
    yi = np.array([])
    for yr in range(2008,2016):
        vg_window = slice('%d-01-15'%(yr+1),'%d-03-01'%(yr+1))
        sfint_window = slice('%d-12-01'%yr,'%d-02-28'%(yr+1))
        xii = vg[v].sel(time=vg_window).mean()        
        yii = (sfint[v].sel(time=sfint_window).groupby('time.month').mean()*np.array((wJ,wF,wD))).sum()
        xi = np.append(xi,xii)
        yi = np.append(yi,yii)
    if 'CO2_OCN' not in v:
        x = np.append(x,xi)
        y = np.append(y,yi)    
    if 'T09' in v: 
        marker = filled_markers[i]
        i += 1
        if i == len(filled_markers): i = 0
    elif v == 'CO2_OCN':
        marker = '*'
    else:
        marker = filled_markers[0]
    ax.plot(xi,yi,'.',label=ds[v].long_name,marker=marker,markersize=4)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels,bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

model = sm.OLS(y, sm.add_constant(x))
fitted = model.fit()

xhat = np.linspace(x.min(),x.max(),50)
yhat = fitted.predict(sm.add_constant(xhat))
yerr = y - fitted.predict(sm.add_constant(x))
xbar = np.mean(x)
n = len(x)
df = n - fitted.df_model - 1
t = stats.t.ppf(1.-0.025,df=df)

print(fitted.params)
print(fitted.summary())
s_err = np.sum(np.power(yerr,2))
ci = t * np.sqrt((s_err/(n-2))*(1.0/n + (np.power((xhat-xbar),2) / 
((np.sum(np.power(xhat,2))) - n*(np.power(xbar,2))))))

ax.plot(xhat,yhat,'-',linewidth=1,color='k')    

ax.axvline(vg_obs.CO2_NOAA-vg_mdl.CO2_FFF-vg_mdl.CO2_LND,color='k')
ax.text(vg_obs.CO2_NOAA-vg_mdl.CO2_FFF-vg_mdl.CO2_LND+0.005,-0.75,'Observed\ngradient\n(corrected for fossil\nand land)')

ax.text(-0.3,-1.4,'{0:0.2f} Pg C yr$^{{-1}}$:ppm\nr$^2$={1:0.3f}'.format(fitted.params[1],fitted.rsquared))

ax.set_xlabel('Vertical CO$_2$ gradient [ppm]')
ax.set_ylabel('Southern Ocean CO$_2$ flux [Pg C yr$^{-1}$]')
plt.savefig(os.path.join(diro['fig'],'calibrate-vertical-gradient-za-iav.pdf'),bbox_extra_artists=(lgd,),bbox_inches='tight')

In [ ]:
plt.plot(vg.CO2_OCN,sfint.CO2_OCN)

In [ ]:
plt.plot(vg.CO2_OCN, vg.O2_OCN)

In [ ]:
vgmon = vg.groupby('time.month').mean().compute()
sfintmon = sfint.groupby('time.month').mean().compute()
vgmon

In [ ]:
nx = np.concatenate(([-1],np.arange(0,12)))

fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(1,1,1)
ax.scatter(vgmon.CO2_OCN,sfintmon.CO2_OCN,marker='o',facecolors='none',edgecolors='none')
p = ax.plot(vgmon.CO2_OCN[nx],sfintmon.CO2_OCN[nx],'-')

for i in range(12):
    t = ax.text(vgmon.CO2_OCN[i],sfintmon.CO2_OCN[i],xcal.month_letter[i],
                verticalalignment='center',
                horizontalalignment='center',
                fontweight='bold',
                color=p[0].get_color(),
                bbox=dict(boxstyle='square,pad=0.3',fc='white',ec='none'))

ax.axhline(0.,color='gray',linewidth=0.5)
ax.axvline(0.,color='gray',linewidth=0.5)

ax.set_xlabel('CO$_2$ vertical gradient [ppm]')
ax.set_ylabel('CO$_2$ flux [Pg C yr$^{-1}$]')
plt.savefig(os.path.join(diro['fig'],'flux-vertical-gradient-climatological-phase-space'))

In [ ]:
nx = np.concatenate(([-1],np.arange(0,12)))

fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(1,1,1)
ax.scatter(vgmon.O2_OCN,sfintmon.O2_OCN,marker='o',facecolors='none',edgecolors='none')
p = ax.plot(vgmon.O2_OCN[nx],sfintmon.O2_OCN[nx],'-')

for i in range(12):
    t = ax.text(vgmon.O2_OCN[i],sfintmon.O2_OCN[i],xcal.month_letter[i],
                verticalalignment='center',
                horizontalalignment='center',
                fontweight='bold',
                color=p[0].get_color(),
                bbox=dict(boxstyle='square,pad=0.3',fc='white',ec='none'))

ax.axhline(0.,color='gray',linewidth=0.5)
ax.axvline(0.,color='gray',linewidth=0.5)

ax.set_xlabel('CO$_2$ vertical gradient [ppm]')
ax.set_ylabel('CO$_2$ flux [Pg C yr$^{-1}$]')
plt.savefig(os.path.join(diro['fig'],'flux-vertical-gradient-climatological-phase-space'))

Plot the regionally integrated flux against the vertical gradient.  The flux is a weighted mean of the Dec, Jan, Feb fluxes (with weights possibly = 0)

In [ ]:
XN2 = 0.7808
XO2 = 0.2095

nx = np.concatenate(([-1],np.arange(0,12)))

fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(1,1,1)
ax.scatter(vgmon.CO2_OCN,vgmon.O2_OCN,marker='o',facecolors='none',edgecolors='none')
p = ax.plot(vgmon.CO2_OCN[nx],vgmon.O2_OCN[nx],'-')

for i in range(12):
    t = ax.text(vgmon.CO2_OCN[i],vgmon.O2_OCN[i],xcal.month_letter[i],
                verticalalignment='center',
                horizontalalignment='center',
                fontweight='bold',
                color=p[0].get_color(),
                bbox=dict(boxstyle='square, pad=0.3', fc='white', ec='none'))

ax.scatter(vgmon.CO2_T09, vgmon.O2_GKA, marker='o', facecolors='none', edgecolors='none')
p = ax.plot(vgmon.CO2_T09[nx], vgmon.O2_GKA[nx],'-')

    
vg_obs_co2 = vg_obs.CO2_NOAA - vg_mdl.CO2_FFF - vg_mdl.CO2_LND
vg_obs_o2 = vg_obs.O2_AO2 - 1.4 * vg_mdl.CO2_FFF / XO2 - 1.1 * vg_mdl.CO2_LND/XO2

p = ax.plot(vg_obs_co2, vg_obs_o2, 'o')
ax.annotate('Obs', 
            xy=(vg_obs_co2, vg_obs_o2), 
            xytext=(vg_obs_co2, vg_obs_o2-0.2),
            color=p[0].get_color(),
            arrowprops=dict(color=p[0].get_color(),arrowstyle='-',linewidth=0.5),
            )

ax.axhline(0.,color='gray',linewidth=0.5)
ax.axvline(0.,color='gray',linewidth=0.5)

ax.set_xlabel('CO$_2$ vertical gradient [ppm]')
ax.set_ylabel('O$_2$ vertical gradient [per meg]')
plt.savefig(os.path.join(diro['fig'],'vertical-gradient-O2-v-CO2-climatological-phase-space'))